In [1]:
import numpy as np
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

from utils import apostrophe_dict, emoticon_dict, short_word_dict

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.callbacks import EarlyStopping  

In [2]:
df = pd.read_excel('отзывы за лето.xls')

In [3]:
df = df.loc[:, df.columns !='Date']
df.head(4)

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
dtypes: int64(1), object(1)
memory usage: 322.9+ KB


In [5]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Предположим, что положительные оценки - это 4 и 5, а 1, 2 и 3 - отрицательные. Соответственно, установим такой порог:

In [6]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)
df['Rating'].value_counts()

1    16724
0     3935
Name: Rating, dtype: int64

In [7]:
max_words = 1000
max_len = 100
num_classes = 1

epochs = 20
batch_size = 512
print_batch_n = 100

In [8]:
def replace_words(text,dict_): 
    output = ''
    
    for word in text.split(' '):
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    
    return output

In [9]:
def preprocess_text(txt):
    txt = str(txt)
    txt = re.sub('[\,]','',txt)
    txt = replace_words(txt, emoticon_dict)
    txt = replace_words(txt, apostrophe_dict)
    txt = replace_words(txt, short_word_dict)
    txt = ''.join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub('\sне', 'не', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split()]

    return " ".join([w for w in txt if len(w)>1])

In [10]:
df['Content_'] = df['Content']

In [11]:
sw = set(get_stop_words('ru'))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [12]:
df['Content'] = df['Content'].apply(preprocess_text)

In [13]:
df_train, df_test = train_test_split(df, test_size=0.25)

In [14]:
train_corpus = ' '.join(df_train['Content'])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

In [15]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [16]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
            
    padding = [0]*(maxlen-len(result))
    
    return padding + result[-maxlen:]

In [17]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test['Content']], dtype=np.int32)

In [18]:
num_classes = 2
y_train = tensorflow.keras.utils.to_categorical(df_train['Rating'], num_classes)
y_test = tensorflow.keras.utils.to_categorical(df_test['Rating'], num_classes)

In [19]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation('relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 2s 64ms/step - loss: 0.5239 - accuracy: 0.8058 - val_loss: 0.3916 - val_accuracy: 0.8232
Epoch 2/20
28/28 [==============================] - 2s 62ms/step - loss: 0.3146 - accuracy: 0.8529 - val_loss: 0.2210 - val_accuracy: 0.9077
Epoch 3/20
28/28 [==============================] - 2s 61ms/step - loss: 0.2100 - accuracy: 0.9112 - val_loss: 0.2045 - val_accuracy: 0.9135
Epoch 4/20
28/28 [==============================] - 2s 61ms/step - loss: 0.1846 - accuracy: 0.9263 - val_loss: 0.2063 - val_accuracy: 0.9194


In [22]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

11/11 [==============================] - 0s 15ms/step


In [23]:
y_pred = np.argmax(results, axis=1)
accuracy_score(df_test['Rating'].to_numpy(), y_pred)

0.9084220716360116